In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv1D, LSTM, CuDNNLSTM, Flatten, Dropout
from keras import optimizers
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
class trainerHelper():
    
    def __init__(self, data, labels, win_size, num_sensor_data, num_classes):
        self.data = data
        self.labels = labels
        self.win_size = win_size
        self.sensor_data = num_sensor_data
        self.num_classes = num_classes
        self.data_length = self.data.shape[0]
        print(self.data_length)
        self.start = 0
        
    def windows(self):
 
        while self.start + self.win_size < self.data_length:
            yield int(self.start), int(self.start + self.win_size)
            self.start += (self.win_size/2)
    
    def segment_data(self):
        self.start = 0
        segments = np.empty((0, self.win_size, self.sensor_data))
        labels= np.empty((0, self.num_classes))
        for (start, end) in self.windows():
            x = np.zeros((1, self.win_size, self.sensor_data))
            x[0,:] = self.data[start:end,:]
            label = self.labels[start:end,:]
            if(x.shape[1] == self.win_size):
                segments = np.vstack([segments,x])
                lb = np.zeros((1, self.num_classes))
                lb[0,:] = label[-1]
                labels = np.vstack([labels,lb])
        return segments, labels

In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

def prepare_data(train_data, test_data):
    encoder = OneHotEncoder()
    train_labels = encoder.fit_transform(train_data['labels'].values.reshape(-1,1)).toarray()
    test_labels = encoder.transform(test_data['labels'].values.reshape(-1,1)).toarray()
    print(train_labels.shape)
    print(test_labels.shape)
    scaler = MinMaxScaler()
    train_data.drop(['labels'], axis=1, inplace=True)
    test_data.drop(['labels'], axis=1, inplace=True)
    train_data = scaler.fit_transform(train_data)
    test_data = scaler.transform(test_data)
    
    return train_data, test_data, train_labels, test_labels

In [4]:
adl_1 = pd.read_csv("ADL1Opportunity_taskB2_S1.csv",header=None)
adl_2 = pd.read_csv("ADL2Opportunity_taskB2_S1.csv",header=None)
adl_3 = pd.read_csv("ADL3Opportunity_taskB2_S1.csv",header=None)
adl_4 = pd.read_csv("ADL4Opportunity_taskB2_S1.csv",header=None)
adl_5 = pd.read_csv("ADL5Opportunity_taskB2_S1.csv",header=None)
drill = pd.read_csv("Drill1Opportunity_taskB2.csv",header=None)

In [5]:
train_frames = [adl_1, adl_2, adl_3, drill]
test_frames = [adl_4, adl_5]
train_data = pd.concat(train_frames)
test_data = pd.concat(test_frames)
train_data.rename(columns ={113: 'labels'}, inplace =True)
test_data.rename(columns ={113: 'labels'}, inplace =True)

In [6]:
print(test_data.shape)
train_data.head()

(63083, 114)


,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,labels
0,148.0,956.0,-358.0,19.0,986.0,196.0,95.0,975.0,152.0,194.0,...,319,-845,-20,57,42,57,20,42,175,1
1,89.0,973.0,-287.0,10.0,1004.0,162.0,125.0,968.0,122.0,224.0,...,325,-847,-17,38,31,38,17,31,175,1
2,130.0,988.0,-418.0,-11.0,1014.0,202.0,127.0,1002.0,113.0,279.0,...,328,-852,27,31,15,31,-27,15,175,1
3,89.0,980.0,-425.0,-47.0,1025.0,191.0,110.0,1006.0,105.0,353.0,...,321,-852,26,22,-2,22,-26,-2,175,1
4,64.0,857.0,-391.0,-8.0,1022.0,204.0,97.0,1002.0,93.0,548.0,...,321,-850,22,45,-7,45,-22,-7,175,1


In [7]:
scaled_train, scaled_test, train_labels, test_labels = prepare_data(train_data, test_data)

(171500, 18)
(63083, 18)


In [8]:
print(scaled_train.shape)
print(train_labels.shape)

(171500, 113)
(171500, 18)


In [10]:
win_size = 15
classes = 18
num_sensors = scaled_train.shape[1]

train_segments = np.empty((0, win_size, num_sensors))
train_segments_labels= np.empty((0, classes))
last_idx = 0
for adl in train_frames:

    trainer_helper = trainerHelper(scaled_train[last_idx:last_idx+adl.shape[0]], train_labels[last_idx:last_idx+adl.shape[0]], win_size, num_sensors, classes)
    segments, labels = trainer_helper.segment_data()
    train_segments = np.vstack([train_segments, segments])
    train_segments_labels = np.vstack([train_segments_labels, labels])
    last_idx = adl.shape[0]

print('Train data has been segmented')

test_segments = np.empty((0, win_size, num_sensors))
test_segments_labels= np.empty((0, classes))
last_idx = 0
for adl in test_frames:

    trainer_helper = trainerHelper(scaled_test[last_idx:last_idx+adl.shape[0]], test_labels[last_idx:last_idx+adl.shape[0]], win_size, num_sensors, classes)
    segments, labels = trainer_helper.segment_data()
    test_segments = np.vstack([test_segments, segments])
    test_segments_labels = np.vstack([test_segments_labels, labels])
    last_idx = adl.shape[0]
    
print('Test data has been segmented')    

51088
32223
33274
54915
Train data has been segmented
32955
30128
Test data has been segmented


In [11]:
train_segments.shape

(22860, 15, 113)

In [12]:
train_segments_labels.shape

(22860, 18)

In [13]:
reshaped_train = train_segments.transpose(0,2,1)
reshaped_test = test_segments.transpose(0,2,1)

In [15]:
model = Sequential()
kernel_height = 5
inputshape = (num_sensors, win_size)


In [16]:
model.add(Conv1D(64, kernel_size=kernel_height, strides=1,
                 activation='relu',
                 input_shape=inputshape))

model.add(Conv1D(64, kernel_size=kernel_height, strides=1,
                 activation='relu'))

model.add(Conv1D(64, kernel_size=kernel_height, strides=1,
                 activation='relu'))

model.add(Conv1D(64, kernel_size=kernel_height, strides=1,
                 activation='relu'))

model.add(Dropout(0.5))

model.add(CuDNNLSTM(128, return_sequences=True))

model.add(Dropout(0.5))

model.add(CuDNNLSTM(128, return_sequences=False))

model.add(Dropout(0.5))

model.add(Dense(classes, activation='softmax'))

rms = optimizers.RMSprop(lr=0.001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [17]:
for layer in model.layers:
    print(layer.name)

conv1d_1
conv1d_2
conv1d_3
conv1d_4
dropout_1
cu_dnnlstm_1
dropout_2
cu_dnnlstm_2
dropout_3
dense_1


In [19]:
batchSize = 100
validationRatio=1.0;
model.fit(reshaped_train,train_segments_labels, validation_split=1-validationRatio,epochs=50,batch_size=batchSize,verbose=1)

print('Calculating score.. ')
score = model.evaluate(reshaped_test,test_segments_labels,verbose=1)
print(score)
model.save('taskB2_S1_CNN_LSTM_model.h5')

Epoch 1/50
22860/22860 [==============================] - 11s 471us/step - loss: 0.1563 - acc: 0.9501
Epoch 2/50
22860/22860 [==============================] - 10s 422us/step - loss: 0.1404 - acc: 0.9550
Epoch 3/50
22860/22860 [==============================] - 10s 421us/step - loss: 0.1348 - acc: 0.9563
Epoch 4/50
22860/22860 [==============================] - 10s 422us/step - loss: 0.1331 - acc: 0.9551
Epoch 5/50
22860/22860 [==============================] - 10s 426us/step - loss: 0.1277 - acc: 0.9573
Epoch 6/50
22860/22860 [==============================] - 10s 423us/step - loss: 0.1227 - acc: 0.9605
Epoch 7/50
22860/22860 [==============================] - 10s 423us/step - loss: 0.1163 - acc: 0.9615
Epoch 8/50
22860/22860 [==============================] - 10s 422us/step - loss: 0.1110 - acc: 0.9643
Epoch 9/50
22860/22860 [==============================] - 10s 422us/step - loss: 0.1091 - acc: 0.9628
Epoch 10/50
22860/22860 [==============================] - 10s 422us/step - loss: 

In [19]:
predictions = model.predict(reshaped_test)

In [20]:
count = 0.0
den = 0.0
for pair in zip(predictions, test_segments_labels):
    if np.argmax(pair[0]) == np.argmax(pair[1]):
        count += 1.0
    den += 1.0

print(count / den)

0.8530876494023905


In [38]:
pred_df = pd.DataFrame(predictions)
pred_df.to_csv('preds_test.csv', header=False, index=False)

In [39]:
true_df = pd.DataFrame(testY)
true_df.to_csv('true_test.csv', header=False, index=False)